In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset

In [7]:


model = smp.Unet(
    encoder_name="resnet34",       
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=1, 
    activation='sigmoid'
)

In [124]:
np.inf > 100

True

In [125]:
from torch.autograd import Variable

In [100]:
import torch
import random
import math
import cv2
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from torchvision import transforms

In [108]:
    
class ValidDataset(Dataset):

    def __init__(self, path, data):
        self.path = path
        data = data.reset_index(drop=True)
        self.image = data['image'].tolist()
        self.label = data['image'].map(lambda x : x.replace(".jpg",".png")).tolist()
        transform = []
        transform.append(A.RandomCrop(width=720, height=720))
        transform.append(A.HorizontalFlip(p=0.5))
        transform.append(A.PadIfNeeded(min_height=736, min_width=736, value=255))
        self.transform = A.Compose(transform)
        return None
    
    def __len__(self):
        return len(self.image)

    def __image__(self, index):  
        image = self.path + '/images/valid/' + self.image[index]
        label = self.path + '/labels/valid/' + self.label[index]
        image = cv2.imread(image)
        label = cv2.imread(label)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        output = self.transform(image=image, mask=label)
        image, label = output['image'], output['mask']
        mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
        transform = []
        transform.append(transforms.ToTensor())
        transform.append(transforms.Normalize(mean, std))
        transform = transforms.Compose(transform)
        image = transform(image)
        label = label.min(axis=-1)
        label = (np.array(label) != 255).astype(float)
        label = torch.from_numpy(label).squeeze()
        return image, label

    def __getitem__(self, idx):
        image, label = self.__image__(idx)
        sample = {'idx': self.image[idx].replace(".jpg",""), 'image': image, 'label': label}
        return sample

In [109]:
data = pd.read_csv('../../data/valid.csv')

In [110]:
x = ValidDataset('../../data/', data)

In [111]:
a = x.__getitem__(0)

In [112]:
a['label'].unique(return_counts=True)

(tensor([0., 1.], dtype=torch.float64), tensor([537742,   3954]))

In [113]:
a['image'].shape

torch.Size([3, 736, 736])

In [114]:
a['label'].min(), a['label'].max()

(tensor(0., dtype=torch.float64), tensor(1., dtype=torch.float64))

In [67]:
images = data['image'].map(lambda x : x.replace(".jpg",".png")).tolist()
images = ['../../data/labels/train/' + x for x in images]

In [79]:
x = np.array(cv2.imread(images[0]))

In [116]:
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp

class Model(nn.Module):
    
    def __init__(self):
        super().__init__()
        params = {}
        params['encoder_name'] = 'resnet34'
        params['encoder_weights'] = 'imagenet'
        params['in_channels'] = 3
        params['classes'] = 1
        params['activation'] = 'identity'
        self.model = smp.Unet(**params)
        return None
    
    def forward(self, image):
        output = self.model(image)
        output = output.squeeze()
        return output

In [117]:
x = Model()

In [120]:
y = x(a['image'].reshape(1,3,736,736))

In [121]:
y.shape

torch.Size([1, 1, 736, 736])

In [80]:
x.shape

(720, 1280, 3)

In [81]:
x = x.min(axis=-1)

In [82]:
x.shape

(720, 1280)

In [ ]:
for image in data['image']